In [ ]:
!pip install streamlit
!pip install google-generativeai
!pip install faiss-cpu
!pip install sentence-transformers
!pip install transformers

In [ ]:
import streamlit as st
import google.generativeai as palm
import faiss
from sentence_transformers import SentenceTransformer

In [ ]:
import google.generativeai as palm

# Set your API key for Google PaLM
API_KEY = "your API_KEY"
palm.configure(api_key=API_KEY)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install -U langchain-community

In [ ]:
# Define the path to the CSV file in your Google Drive
file_path = '/content/drive/MyDrive/ADVAI/codebasics_faqs.csv'

# Load the CSV file using CSVLoader with a specific encoding
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path=file_path, source_column='prompt', encoding='ISO-8859-1')  # Use ISO-8859-1 or 'latin1' encoding
data = loader.load()

# Display the loaded data to confirm it's working
print(data)

In [ ]:
import pandas as pd

# Load the dataset (replace with your file path if needed)
faq_data = pd.read_csv('/content/drive/MyDrive/ADVAI/codebasics_faqs.csv', encoding='ISO-8859-1')

# Ensure columns exist
assert 'prompt' in faq_data.columns and 'response' in faq_data.columns, "Dataset must have 'question' and 'answer' columns."

# Extract questions and answers
questions = faq_data['prompt'].tolist()
answers = faq_data['response'].tolist()

In [ ]:
from sentence_transformers import SentenceTransformer

# Load HuggingFace Instructor Model
embedding_model = SentenceTransformer('hkunlp/instructor-xl',device='cpu')

# Create embeddings for the FAQ questions
question_embeddings = embedding_model.encode(questions, convert_to_tensor=True)

In [ ]:
import faiss
import numpy as np

# Initialize FAISS index
embedding_dim = question_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Convert to numpy array and add to FAISS index
question_embeddings_np = question_embeddings.detach().cpu().numpy()
faiss_index.add(question_embeddings_np)

In [ ]:
!pip install pyngrok

In [ ]:
app_code = """
import streamlit as st
import google.generativeai as palm
import faiss
from sentence_transformers import SentenceTransformer
import pandas as pd
from datetime import datetime

# Load the dataset
faq_data = pd.read_csv('/content/drive/MyDrive/ADVAI/codebasics_faqs.csv', encoding='ISO-8859-1')

# Ensure columns exist
assert 'prompt' in faq_data.columns and 'response' in faq_data.columns, "Dataset must have 'prompt' and 'response' columns."

# Extract questions and answers
questions = faq_data['prompt'].tolist()
answers = faq_data['response'].tolist()

# Load the embedding model
embedding_model = SentenceTransformer('hkunlp/instructor-xl')

# Create FAISS index
embedding_dim = embedding_model.get_sentence_embedding_dimension()
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Create embeddings for the FAQ prompts
question_embeddings = embedding_model.encode(questions, convert_to_tensor=True)
faiss_index.add(question_embeddings.detach().cpu().numpy())

# Configure Google PaLM
palm.configure(api_key="your-google-palm-api-key")

# Feedback storage (e.g., CSV or a database)
FEEDBACK_FILE = "feedback.csv"

# Function to save feedback
def save_feedback(query, response, feedback):
    feedback_data = {
        "query": query,
        "response": response,
        "feedback": feedback,
        "timestamp": datetime.now().isoformat()
    }
    feedback_df = pd.DataFrame([feedback_data])

    # Append feedback to CSV
    try:
        feedback_df.to_csv(FEEDBACK_FILE, mode='a', header=not open(FEEDBACK_FILE).readlines(), index=False)
    except FileNotFoundError:
        feedback_df.to_csv(FEEDBACK_FILE, index=False)
    st.success("Feedback submitted successfully!")

# Streamlit app UI
st.title("FAQ Assistant with Generative AI")

query = st.text_input("Ask a question:")

if query:
    # Generate embedding for the query
    query_embedding = embedding_model.encode([query], convert_to_tensor=True).detach().cpu().numpy()

    # Search for the closest match in FAISS
    D, I = faiss_index.search(query_embedding, k=1)
    best_match_index = I[0][0]
    similarity_score = D[0][0]

    # Define a similarity threshold
    similarity_threshold = 0.5

    if similarity_score < similarity_threshold:
        # Fetch and display the corresponding response
        answer = answers[best_match_index]
        st.write(f"**Answer:** {answer}")
    else:
        # Display a fallback response if no good match is found
        answer = "I don't know the answer to this question. Please try rephrasing."
        st.write(answer)

    # Feedback buttons
    st.write("Was this response helpful?")
    col1, col2, col3 = st.columns(3)
    with col1:
        if st.button("Good"):
            save_feedback(query, answer, "Good")
    with col2:
        if st.button("Neutral"):
            save_feedback(query, answer, "Neutral")
    with col3:
        if st.button("Bad"):
            save_feedback(query, answer, "Bad")

# Optional: Display feedback data
if st.checkbox("View Feedback"):
    try:
        feedback_df = pd.read_csv(FEEDBACK_FILE)
        st.write(feedback_df)
    except FileNotFoundError:
        st.write("No feedback available yet.")
"""

# Save the updated code to `app.py`
with open("app.py", "w") as file:
    file.write(app_code)

print("Streamlit app saved as `app.py` with feedback functionality.")


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

# Load the dataset
faq_data = pd.read_csv('/content/drive/MyDrive/ADVAI/codebasics_faqs.csv', encoding='ISO-8859-1')

# Ensure columns exist
assert 'prompt' in faq_data.columns and 'response' in faq_data.columns, "Dataset must have 'prompt' and 'response' columns."

# Extract questions and answers
questions = faq_data['prompt'].tolist()
answers = faq_data['response'].tolist()

# Load the embedding model
embedding_model = SentenceTransformer('hkunlp/instructor-xl')

# Create embeddings for the FAQ prompts
print("Generating embeddings for FAQ questions...")
question_embeddings = embedding_model.encode(questions, convert_to_tensor=False)

# Display embeddings for each question
print("FAQ Question Embeddings:")
for i, (question, embedding) in enumerate(zip(questions, question_embeddings)):
    print(f"Question {i+1}: {question}")
    print(f"Embedding: {embedding[:5]}... (truncated for display)")  # Display first 5 values for readability
    print("-" * 50)

# Create FAISS index
embedding_dim = question_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to FAISS index
question_embeddings_np = question_embeddings
faiss_index.add(question_embeddings_np)

# Simulate a query and display its embedding
query = "How do I reset my password?"
print("\nUser Query:", query)

# Generate embedding for the query
query_embedding = embedding_model.encode([query], convert_to_tensor=False)

# Display query embedding
print("Query Embedding:")
print(query_embedding[0][:5], "... (truncated for display)")

# Search for the closest match in FAISS
D, I = faiss_index.search(query_embedding, k=1)
best_match_index = I[0][0]
similarity_score = D[0][0]

# Display the closest match and similarity score
print("\nClosest Match:")
print(f"FAQ Question: {questions[best_match_index]}")
print(f"Similarity Score: {similarity_score}")


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('hkunlp/instructor-xl')  # Lightweight model
expected = "No"
predicted = "No"

# Get embeddings
expected_embedding = model.encode(expected, convert_to_tensor=True)
predicted_embedding = model.encode(predicted, convert_to_tensor=True)

# Calculate cosine similarity
similarity = util.pytorch_cos_sim(expected_embedding, predicted_embedding)
print(f"Similarity Score: {similarity.item():.2f}")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
file_path = '/content/drive/MyDrive/ADVAI/codebasics_faqs.csv'  # Replace with your file path
faq_data = pd.read_csv(file_path, encoding='ISO-8859-1')

# Check for missing values
print("Missing values per column:")
print(faq_data.isnull().sum())

# Set up the heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(faq_data.isnull(), cbar=False, cmap="viridis", yticklabels=False)

# Add labels and title
plt.title("Heatmap of Missing Data")
plt.xlabel("Columns in Dataset")
plt.ylabel("Rows")

# Show the heatmap
plt.show()


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk

# Ensure NLTK data is available
nltk.download('punkt')

# Example test set
test_queries = ["How do I reset my password?", "How to update my profile?"]
expected_answers = ["You can reset your password by clicking on 'Forgot Password'.",
                    "To update your profile, go to settings."]

# Iterate through test queries
for i, test_query in enumerate(test_queries):
    # Generate query embedding
    query_embedding = embedding_model.encode([test_query], convert_to_tensor=False)

    # Search for closest match
    D, I = faiss_index.search(query_embedding, k=1)
    best_match_index = I[0][0]
    predicted_answer = answers[best_match_index]

    # Calculate BLEU score
    reference = nltk.word_tokenize(expected_answers[i].lower())
    candidate = nltk.word_tokenize(predicted_answer.lower())
    smoothie = SmoothingFunction().method4  # Smoothing for short sentences
    bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothie)

    # Print results
    print(f"Query: {test_query}")
    print(f"Expected Answer: {expected_answers[i]}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"BLEU Score: {bleu_score:.2f}")
    print("-" * 50)


In [ ]:
!pip install pyngrok


In [ ]:
!ngrok authtoken "your auth token"

In [ ]:
from pyngrok import ngrok

# Start ngrok with explicit port configuration
public_url = ngrok.connect(8051)
print("Streamlit app is running. Access it via:", public_url)

# Run the Streamlit app
!streamlit run app.py &>/dev/null &


In [ ]:
from pyngrok import ngrok
import subprocess

# Start ngrok with explicit port configuration
public_url = ngrok.connect(8051)
print("Streamlit app is running. Access it via:", public_url)

# Run the Streamlit app
subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8051"])
